In [1]:
import pandas as pd

In [2]:
file_path = r"C:\Users\Rajat Saini\OneDrive\Desktop\mrs\archive\spotify_millsongdata.csv"
df = pd.read_csv(file_path)

In [3]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [4]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [5]:
df.shape

(57650, 4)

In [6]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [7]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [8]:
df.head(10)

,artist,song,text
0,Religious Music,"Come Rejoicing, Praises Voicing","Come rejoicing, praises voicing \r\nChristmas..."
1,Hooverphonic,Autoharp,You are my autoharp \r\nI push every button o...
2,Hank Williams Jr.,How Long Will You Keep Coming Back To Me,I know it's only natural \r\nIt's just the ma...
3,NOFX,Creeping Out Sara,I was backstage at a festival in Germany \r\n...
4,Don Moen,Yo Deseo Estar Donde Tu Estas,Yo deseo estar \r\nDonde tu estas \r\nA diar...
5,Neil Diamond,Dancing In The Street,(are you ready to let it happen to you tonight...
6,Vanilla Ice,Ice Is Workin' It,Yo! here comes the intro \r\nIt's time to let...
7,Amy Grant,"Emmanuel, God With Us",We dim the light. \r\nWe stoke the fire. \r\...
8,Bob Seger,It's You,Just about the time I think I've had it \r\nE...
9,Bruce Springsteen,Child Bride,"A lovely, poignant song \r\nAbout a man fated..."


In [9]:
df['text'][0]

"Come rejoicing, praises voicing  \r\nChristmas Day is breaking  \r\nNow th'Eternal, Lord supernal  \r\nHuman form is taking  \r\nOn the hay - lo, behold  \r\nVirgin's Son, as foretold  \r\nLies the precious Infant in the wintry cold  \r\nLies the precious Infant in the wintry cold.  \r\n  \r\nUnabated for Him waited  \r\nMany generations  \r\nHim now proudly angels loudly  \r\nPraise the Hope of nations  \r\nLet us, then, gladly sing  \r\nLet our songs gaily ring  \r\nAs we to this holy Child our praises bring  \r\nAs we to this holy Child our praises bring.  \r\n  \r\nTender Flower, Mighty Tower  \r\nJesus Christ, our Savior  \r\nHeav'nly Treasure, without measure  \r\nThee we love forever  \r\nLord and King without end  \r\nOur poor hearts now befriend  \r\nAll Thy gifts of grace and goodness to us send  \r\nAll Thy gifts of grace and goodness to us send...\r\n\r\n"

In [10]:
# df = df.sample(5000)

In [11]:
df.shape

(5000, 3)

Text Cleaning/ Text Preprocessing

In [12]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [13]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [14]:

df['text'] = df['text'].apply(lambda x: tokenization(x))

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [16]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [17]:
similarity[0]

array([1.        , 0.00732616, 0.02764382, ..., 0.00283086, 0.00729324,
       0.02030632])

In [18]:
df[df['song'] == 'Get A Life']

,artist,song,text


In [24]:
def recommendation(song_df):
    song_indices = df[df['song'] == song_df].index
    if len(song_indices) == 0:
        print("Song not found in DataFrame.")
        return []

    idx = song_indices[0]  # Get the index of the first occurrence of the song

    distances = sorted(list(enumerate(similarity[idx])), reverse=True, key=lambda x: x[1])

    songs = []
    # Adjust the slice here to get 15 recommendations, excluding the input song
    for m_id in distances[1:16]:  # Skip the first one since it is the queried song itself
        songs.append(df.iloc[m_id[0]].song)

    return songs


In [28]:
recommendation('Get Back')

['Get Back',
 'Back Where You Belong',
 'The Fighter',
 'I Belong To You',
 'Flinstone Boy',
 'You Belong To Me',
 'Scent Of A Woman',
 'Belong To The World',
 'Who Can See It',
 'Long Ago',
 'Twisted',
 'Show Me',
 'Once Upon A Time',
 'Around The Plynth',
 'Once A Day']

In [26]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))

In [27]:
import pandas as pd
import numpy as np

# Assuming df.pkl is a DataFrame
df = pd.read_pickle('C:\\Users\\Rajat Saini\\df.pkl')
df.to_csv('C:\\Users\\Rajat Saini\\music.csv', index=False)

# Assuming similarity.pkl is a numpy array
similarity = pd.read_pickle('C:\\Users\\Rajat Saini\\similarity.pkl')
np.save('C:\\Users\\Rajat Saini\\similarity.npy', similarity)
